In [1]:
import os
import librosa
import numpy as np
from scipy.ndimage import maximum_filter

def get_audio_files_from_dir(directory, extensions={".wav", ".mp3"}):
    return [
        os.path.join(directory, f)
        for f in os.listdir(directory)
        if os.path.splitext(f)[1].lower() in extensions
    ]

def get_spectrogram_peaks(y, sr, n_fft=4096, hop_length=512, threshold_db=-30):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    S_db = librosa.amplitude_to_db(S, ref=np.max)
    
    # Local maxima detection
    neighborhood_size = (20, 20)
    local_max = (S_db == maximum_filter(S_db, footprint=np.ones(neighborhood_size)))
    peaks = np.argwhere(local_max & (S_db > threshold_db))  # (freq_bin, time_bin)

    return peaks

def generate_peak_pairs(peaks, fanout=5, time_window=50):
    pairs = []
    for i in range(len(peaks)):
        freq1, t1 = peaks[i]
        for j in range(1, fanout):
            if i + j < len(peaks):
                freq2, t2 = peaks[i + j]
                delta_t = t2 - t1
                if 0 < delta_t <= time_window:
                    pairs.append((freq1, freq2, delta_t, t1))
    return pairs

def process_audio_directory(directory_path):
    all_fingerprints = []
    audio_files = get_audio_files_from_dir(directory_path)

    for audio_path in audio_files:
        try:
            filename = os.path.basename(audio_path)
            y, sr = librosa.load(audio_path, sr=None, mono=True)
            peaks = get_spectrogram_peaks(y, sr)
            peak_pairs = generate_peak_pairs(peaks)

            for (freq1, freq2, delta_t, anchor_time) in peak_pairs:
                all_fingerprints.append((filename, int(freq1), int(freq2), int(delta_t), int(anchor_time)))
        except Exception as e:
            print(f"[ERROR] Skipping {audio_path}: {e}")

    return all_fingerprints

Extract Full Song Features

In [29]:
directory_path = "songs_mp3"  # Change this to your folder path if needed

print(f"Processing songs in: {directory_path}")
all_fingerprints = process_audio_directory(directory_path)

Processing songs in: songs_mp3


In [ ]:
import pandas as pd

df = pd.DataFrame(
    all_fingerprints,
    columns=["filename", "freq1", "freq2", "delta_time", "anchor_time"]
)

# Preview results
df[df['filename']]

,filename,freq1,freq2,delta_time,anchor_time
0,Billie Eilish - BIRDS OF A FEATHER (Official M...,3,3,21,17784
1,Billie Eilish - BIRDS OF A FEATHER (Official M...,3,3,20,18210
2,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,20,15066
3,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,37,15903
4,Billie Eilish - BIRDS OF A FEATHER (Official M...,4,4,39,17654
...,...,...,...,...,...
14417,Teddy Swims - Lose Control (The Village Sessio...,481,483,15,18785
14418,Teddy Swims - Lose Control (The Village Sessio...,507,508,13,7076
14419,Teddy Swims - Lose Control (The Village Sessio...,508,509,13,18889
14420,Teddy Swims - Lose Control (The Village Sessio...,523,530,24,18795


In [23]:
df.to_csv("test/Flattened_Fingerprints.csv", index=False)

Extract Clip Fingerprints

In [33]:
directory_path = "clip_mp3"  # Change this to your folder path if needed

print(f"Processing songs in: {directory_path}")
all_clip_fingerprints = process_audio_directory(directory_path)

Processing songs in: clip_mp3


In [34]:
import pandas as pd

df = pd.DataFrame(
    all_clip_fingerprints,
    columns=["filename", "freq1", "freq2", "delta_time", "anchor_time"]
)

# Preview results
df

,filename,freq1,freq2,delta_time,anchor_time
0,clip_Chappell Roan - Pink Pony Club (Official ...,9,14,28,56
1,clip_Chappell Roan - Pink Pony Club (Official ...,20,20,25,683
2,clip_Chappell Roan - Pink Pony Club (Official ...,20,21,45,708
3,clip_Chappell Roan - Pink Pony Club (Official ...,20,21,14,783
4,clip_Chappell Roan - Pink Pony Club (Official ...,21,21,44,753
...,...,...,...,...,...
1244,clip_Teddy Swims - Lose Control (The Village S...,319,320,15,210
1245,clip_Teddy Swims - Lose Control (The Village S...,342,396,12,414
1246,clip_Teddy Swims - Lose Control (The Village S...,401,440,6,36
1247,clip_Teddy Swims - Lose Control (The Village S...,583,594,1,901


In [32]:
df['filename'].value_counts()

filename
clip_Billie Eilish - BIRDS OF A FEATHER (Official Music Video).mp3          7683750
clip_ROSÉ & Bruno Mars - APT. (Official Music Video).mp3                       266
clip_Kendrick Lamar - tv off (Official Audio).mp3                               183
clip_Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video).mp3        151
clip_PARTYNEXTDOOR & DRAKE - NOKIA (Official Lyric Video).mp3                   149
clip_Kendrick Lamar - Not Like Us.mp3                                           128
clip_Teddy Swims - Lose Control (The Village Sessions).mp3                      118
clip_Kendrick Lamar - luther (Official Audio).mp3                               105
clip_Chappell Roan - Pink Pony Club (Official Music Video).mp3                   76
clip_Shaboozey - A Bar Song (Tipsy) [Official Visualizer].mp3                    73
Name: count, dtype: int64

In [35]:
df.to_csv("test/Clip_Flattened_Fingerprints.csv", index=False)